In [1]:
#Step 1: Import Required Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import joblib


In [3]:
#Step 2: Load and Explore Data

In [ ]:
data = pd.read_csv("/content/adult 3.csv")

# Basic info
print(data.head(10))
print(data.shape)
print(data.tail(3))


In [5]:
#Step 3: Handle Missing and Unknown Values

In [ ]:
# Checking missing values
print(data.isna().sum())

# Replace '?' with 'Others' for workclass and occupation
data['workclass'].replace({'?': 'Others'}, inplace=True)
data['occupation'].replace({'?': 'Others'}, inplace=True)

# Filter unwanted workclass categories
data = data[~data['workclass'].isin(['Without-pay', 'Never-worked'])]

# View updated counts
print(data['workclass'].value_counts())
print(data['occupation'].value_counts())


In [7]:
#Step 4: Explore and Remove Outliers

In [ ]:
# Age
plt.boxplot(data['age']); plt.title("Age"); plt.show()
data = data[(data['age'] >= 17) & (data['age'] <= 75)]
plt.boxplot(data['age']); plt.title("Cleaned Age"); plt.show()

# Capital Gain
plt.boxplot(data['capital-gain']); plt.title("Capital Gain"); plt.show()

# Educational Num
plt.boxplot(data['educational-num']); plt.title("Educational Number"); plt.show()
data = data[(data['educational-num'] >= 5) & (data['educational-num'] <= 16)]
plt.boxplot(data['educational-num']); plt.title("Cleaned Educational Num"); plt.show()

# Hours Per Week
plt.boxplot(data['hours-per-week']); plt.title("Hours per Week"); plt.show()


In [9]:
#Step 5: Feature Cleanup

In [10]:
# Drop redundant column
data.drop(columns=['education'], inplace=True)


In [11]:
#Step 6: Label Encoding

In [12]:
encoder = LabelEncoder()

# Apply encoding
categorical_cols = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
for col in categorical_cols:
    data[col] = encoder.fit_transform(data[col])


In [13]:
#Step 7: Split Features and Labels

In [14]:
X = data.drop(columns=['income'])
y = data['income']


In [15]:
#Step 8: Train & Compare Models using Pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "LogisticRegression": LogisticRegression(),
    "RandomForest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "GradientBoosting": GradientBoostingClassifier()
}

results = {}

for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))


In [17]:
#Step 9: Visualize Model Comparison

In [ ]:
plt.bar(results.keys(), results.values(), color='skyblue')
plt.ylabel('Accuracy Score')
plt.title('Model Comparison')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


In [19]:
#Step 10: Final Model Selection and Saving

In [ ]:
# Train and save the best model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Re-train models
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    results[name] = accuracy_score(y_test, preds)
    print(f"{name}: {results[name]:.4f}")

best_model_name = max(results, key=results.get)
best_model = models[best_model_name]

print(f"\n✅ Best model: {best_model_name} with accuracy {results[best_model_name]:.4f}")
joblib.dump(best_model, "best_model.pkl")
print("✅ Saved best model as best_model.pkl")


In [21]:
# Step 11: Streamlit App (Save as app.py)

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

model = joblib.load("best_model.pkl")

st.set_page_config(page_title="Employee Salary Classification", page_icon="💼", layout="centered")
st.title("💼 Employee Salary Classification App")
st.markdown("Predict whether an employee earns >50K or ≤50K based on input features.")

st.sidebar.header("Input Employee Details")

# Input fields (adjust based on training features)
age = st.sidebar.slider("Age", 18, 65, 30)
education = st.sidebar.selectbox("Education Level", [
    "Bachelors", "Masters", "PhD", "HS-grad", "Assoc", "Some-college"
])
occupation = st.sidebar.selectbox("Job Role", [
    "Tech-support", "Craft-repair", "Other-service", "Sales",
    "Exec-managerial", "Prof-specialty", "Handlers-cleaners", "Machine-op-inspct",
    "Adm-clerical", "Farming-fishing", "Transport-moving", "Priv-house-serv",
    "Protective-serv", "Armed-Forces"
])
hours_per_week = st.sidebar.slider("Hours per week", 1, 80, 40)
experience = st.sidebar.slider("Years of Experience", 0, 40, 5)

# Create input DataFrame
input_df = pd.DataFrame({
    'age': [age],
    'education': [education],
    'occupation': [occupation],
    'hours-per-week': [hours_per_week],
    'experience': [experience]
})

st.write("### 🔎 Input Data")
st.write(input_df)

# Predict button
if st.button("Predict Salary Class"):
    prediction = model.predict(input_df)
    st.success(f"✅ Prediction: {prediction[0]}")

# Batch prediction
st.markdown("---")
st.markdown("#### 📂 Batch Prediction")
uploaded_file = st.file_uploader("Upload a CSV file for batch prediction", type="csv")

if uploaded_file is not None:
    batch_data = pd.read_csv(uploaded_file)
    st.write("Uploaded data preview:", batch_data.head())
    batch_preds = model.predict(batch_data)
    batch_data['PredictedClass'] = batch_preds
    st.write("✅ Predictions:")
    st.write(batch_data.head())
    csv = batch_data.to_csv(index=False).encode('utf-8')
    st.download_button("Download Predictions CSV", csv, file_name='predicted_classes.csv', mime='text/csv')
